In [1]:
import A1_provider_prep
import A2_other
import A3_analysis
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.wkt
import shared_utils
from calitp.sql import to_snakecase
from shared_utils import geography_utils, utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# Load in original geometries of the routes.
one_dist_routes, multi_dist_routes, all_routes = A2_other.find_multi_district_routes()

In [7]:
agencies = all_routes[["agency", "itp_id"]].drop_duplicates().reset_index(drop=True)

In [16]:
len(agencies)

168

In [ ]:
subset_gtfs = ["itp_id", "name", "gtfs_static_status", "gtfs_realtime_status"]

In [62]:
def load_gtfs(): 
    
    # Downloaded from Airtable "Organizations"
    # https://airtable.com/appPnJWrQ7ui4UmIl/tblFsd8D5oFRqep8Z/viwVBVSd0ZhYu8Ewm?blocks=hide
    subset_gtfs = ["itp_id", "name", "gtfs_static_status", "gtfs_realtime_status"]
    
    df = to_snakecase(
    pd.read_csv(f"{A1_provider_prep.GCS_FILE_PATH}airtable_organizations.csv"))[subset_gtfs]
    
    df.itp_id = df.itp_id.fillna(0).astype(int)
    
    df.name = df.name.replace(
    {"Eastern Sierra Transit Authority": "Mammoth Lakes Transit System"})
    
    # Consolidate GTFS into one col
    df["gtfs_status"] = df.gtfs_static_status + '/' + m1.gtfs_realtime_status
    
    return df 

In [60]:
test = add_gtfs()

In [53]:
    m1 = agencies.merge(gtfs, how="left", on=["itp_id"])
    
    # Consolidate GTFS into one col
    m1["gtfs_status"] = m1.gtfs_static_status + '/' + m1.gtfs_realtime_status
    
    return m1

In [14]:
gtfs = to_snakecase(
    pd.read_csv(f"{A1_provider_prep.GCS_FILE_PATH}airtable_organizations.csv")
)[subset_gtfs]

In [19]:
gtfs.itp_id = gtfs.itp_id.fillna(0).astype(int)

In [43]:
gtfs.name = gtfs.name.replace(
    {"Eastern Sierra Transit Authority": "Mammoth Lakes Transit System"}
)

In [44]:
len(gtfs)

906

In [54]:
m1 = agencies.merge(gtfs, how="outer", on=["itp_id"], indicator = True)

In [55]:
m1.loc[m1._merge == "left_only"]

,agency,itp_id,name,gtfs_static_status,gtfs_realtime_status,_merge
8,Avocado Heights/Bassett/West Valinda Shuttle,171,NaN,NaN,NaN,left_only
28,Clean Air Express,473,NaN,NaN,NaN,left_only
29,Cloverdale Transit,70,NaN,NaN,NaN,left_only
41,Dumbarton Express,98,NaN,NaN,NaN,left_only
42,East Los Angeles Shuttle,172,NaN,NaN,NaN,left_only
43,East Valinda Shuttle,173,NaN,NaN,NaN,left_only
67,Irvine Shuttle,142,NaN,NaN,NaN,left_only
82,Mammoth Lakes Transit System,190,NaN,NaN,NaN,left_only
137,Sunshine Bus(South Whittier),174,NaN,NaN,NaN,left_only
167,the Link Florence-Firestone/Walnut Park,177,NaN,NaN,NaN,left_only


In [50]:
m1["gtfs_status"] = m1.gtfs_static_status + '/' + m1.gtfs_realtime_status

In [51]:
# gtfs.name.sort_values().tolist()

In [52]:
m1.sample()

,agency,itp_id,name,gtfs_static_status,gtfs_realtime_status,gtfs_status
63,Grapeline,168,City of Lodi,Static OK,RT Incomplete,Static OK/RT Incomplete
